In [1]:
require 'torch'
require 'nn'
require 'cunn'
require 'cudnn'

local activation = nn.ReLU

local function inference_block(opt)
-- X, Q_v', Q_o' = unpack(net:forward(X, Q_o, Q_v))

    local nInputX = opt.nFeatures or 4096
    local nInputQ_o = opt.nObjects or 38
    local nInputQ_v = opt.nVerbs or 33
    
    local function basic(nInput1, nInput2, nOutput)
        local net = nn.Sequential()
            :add(nn.ParallelTable()
                :add(nn.Linear(nInput1, nOutput))
                :add(nn.Linear(nInput2, nOutput)))
            :add(nn.CAddTable(true))
            :add(nn.BatchNormalization(nOutput))
            :add(activation())
        return net
    end
    
    local net = nn.Sequential()
        :add(nn.ConcatTable() 
            :add(nn.SelectTable(1))
            :add(nn.ConcatTable()
                :add(nn.Sequential()
                    :add(nn.ConcatTable()
                        :add(nn.SelectTable(1))
                        :add(nn.SelectTable(3)))
                    :add(basic(nInputX, nInputQ_v, nInputQ_o)))
                :add(nn.Sequential()
                    :add(nn.NarrowTable(1,2))
                    :add(basic(nInputX, nInputQ_o, nInputQ_v)))))
        :add(nn.FlattenTable())
    return net
end

function createModel(opt)
    local nUnits = opt.unitNum or 8
    local inputSize = opt.nObjects + opt.nVerbs
    local lstmOutputSize = 256

    local model = nn.Sequential()
    
    -- Inference units
    local unit = inference_block(opt)
    for i=1, nUnits do
        if opt.share then
            model:add(unit)
        else
            model:add(inference_block(opt))
        end
    end
    model:add(nn.NarrowTable(2,2))
    model:add(nn.JoinTable(2))

    -- LSTM layer
    local lstm = cudnn.LSTM(inputSize, lstmOutputSize, 1, true) 
    model:add(nn.View(opt.batchSize, opt.timesteps, -1))
    model:add(nn.Copy(nil, nil, true))
    model:add(lstm)
    -- Dropout layer
    if opt.dropout > 0 then 
        model:add(nn.Dropout(opt.dropout))
    end
    -- Last FC layer
    model:add(nn.View(-1, lstmOutputSize))
    model:add(nn.Linear(lstmOutputSize, opt.nClasses))
    model:cuda()
    
--     print(tostring(model))

    return model
end

In [11]:
opt = {
    share = true,
    unitNum = 8,
    nObjects = 50,
    nFeatures = 256,
    nVerbs = 35,
    nClasses = 157,
    dropout = 0.5,
    batchSize = 32,
    timesteps = 16
}
-- X = torch.rand(opt.batchSize * opt.timesteps, opt.nFeatures)
-- Y = torch.rand(opt.batchSize * opt.timesteps, opt.nObjects)
-- Z = torch.rand(opt.batchSize * opt.timesteps, opt.nVerbs)

-- model = createModel(opt):double()
-- input = {X, Y, Z}
-- pred = model:forward(input)
-- print(pred:type())
pred = torch.rand(5, 10)
print(pred)


target = torch.rand(5, 10) + 0.5
target = target:int():double()
print(target)

criterion = nn.MultiLabelSoftMarginCriterion()
err = criterion:forward(pred, target)
print(err)

-- print(model.modules[1].modules[1].modules[2].modules[2].modules[2].modules[1].modules[2].weight)

-- generateGraph = require 'optnet.graphgen'

-- -- visual properties of the generated graph
-- -- follows graphviz attributes
-- graphOpts = {
-- displayProps =  {shape='ellipse',fontsize=14, style='solid'},
-- nodeData = function(oldData, tensor)
--   return oldData .. '\n' .. 'Size: '.. tensor:numel()
-- end
-- }

-- g = generateGraph(model, input, graphOpts)

-- graph.dot(g,'SRT_Net','no_share')

 0.4352  0.4800  0.5321  0.0594  0.4484  0.8854  0.3946  0.4259  0.9592  0.0969
 0.8376  0.0745  0.2013  0.3689  0.1984  0.6971  0.5427  0.0306  0.3242  0.9093
 0.7759  0.1776  0.0250  0.9952  0.9585  0.3937  0.7873  0.4887  0.4268  0.3051
 0.7661  0.4334  0.3017  0.1215  0.3667  0.2899  0.9170  0.6679  0.2521  0.9375
 0.9233  0.4054  0.0360  0.8611  0.9246  0.4254  0.5632  0.2845  0.9418  0.6647
[torch.DoubleTensor of size 5x10]

 0  0  1  1  0  1  0  1  0  0
 1  0  1  1  0  1  1  1  0  0
 1  0  0  0  0  1  0  0  0  1
 1  1  0  0  1  1  1  1  0  0
 1  0  0  0  1  1  0  0  1  0
[torch.DoubleTensor of size 5x10]

0.73418871899445	


In [44]:
print(model.modules[2].modules[1].modules[2].modules[2].modules[2].modules[1].modules[2].weight)

Columns 1 to 6
 3.9041e-01 -3.3979e-01 -7.5614e-01  7.2086e-02  1.5265e-02 -9.2803e-01
-1.3880e+00 -6.6663e+00 -2.8577e+00  2.4739e+00 -3.5433e-02  3.8981e-01
 1.0026e-01 -4.4865e-01  7.4888e-01 -2.6959e-02 -4.9950e-02  7.1962e-01
 1.4032e+00 -6.5603e-01 -2.7301e+00 -3.9351e+00  1.3820e-01  4.5879e+00
-1.2890e+00  4.7842e-01  3.9974e+00 -1.6127e+00 -7.3735e-02 -2.1290e+00
-2.0506e-01 -1.9487e-01 -7.9634e-01 -5.3255e-01  6.2478e-02 -4.1837e-01
-2.9595e-01  5.4872e-02  4.9842e-01 -1.5785e-01  4.6645e-02 -1.1416e-01
-1.1940e-01 -3.5009e+00  8.7417e-01  5.7442e-04 -2.6510e-02  2.4359e+00
-6.2859e-01  1.8430e+00 -3.4067e+00 -2.6288e-01  5.0829e-02  1.7752e+00
-1.0158e+00  1.0555e+00  2.4891e-01 -1.1338e+00 -6.5099e-02 -1.1359e+00
-3.2217e+00 -9.5486e-02  6.5827e+00  1.4674e+00  3.3681e-01 -1.6393e+00
-7.8729e-01  7.9186e+00  3.0841e+00 -1.4205e+00 -1.1829e-02 -1.1903e+00
-1.7594e+00  6.7508e-01  2.7261e+00 -2.1281e-01 -1.0946e-01  1.0510e+00
 8.8542e-01 -4.1810e+00 -5.8989e+00  8.1039e-01  

-01  1.1901e+00  1.9930e+00  1.4540e+00
 8.3141e-02  6.9628e-01  1.0368e-01 -4.5739e-01 -1.2975e+00  4.2980e-01
-1.1709e-01 -2.2776e-01 -7.9894e-02 -2.0416e-01 -1.3894e-01  1.8865e-01
-1.2836e-02  1.2563e+00 -1.6092e-01 -1.0368e+00 -1.3058e-01 -1.6596e+00
 6.3829e-02  1.9705e-01 -1.3030e-01 -1.3146e-01  1.6558e-01 -4.1513e-02
 7.6251e-02  3.6425e+00 -7.9821e-02  2.1050e+00  7.8503e-01 -2.3615e+00
-8.9202e-02  6.5706e-01  5.0711e-01  2.7451e-01  1.1465e+00 -1.5234e+00
-1.9312e-02 -4.6543e-01  2.5093e-02  7.4271e-01 -4.6938e-01 -4.9182e-01
-1.0954e-01 -2.0692e+00  9.3045e-02 -2.8721e-01 -5.0421e-01 -1.0616e+00
 5.6192e-02 -4.4607e-01  3.5268e-02 -6.5109e-02  7.7085e-02  1.7428e-01
-4.5575e-04  1.3964e+00 -2.6514e-02 -2.1125e+00 -2.9032e+00 -5.0424e+00
-7.9939e-02 -1.2527e-01 -4.0889e-04 -5.4256e-01  1.5809e-02 -7.7009e-01
 1.1956e-01  5.7024e-01 -8.6934e-02 -1.0584e-01 -6.5961e-01  1.0976e-01
 3.1103e-02  3.2955e-01  9.4373e-02 -3.7440e-01  2.5633e-01  1.6032e+00

Columns 13 to 18
-4.569

 2.0418e-01
 1.8085e+00 -1.0953e+00  1.9633e-01  6.7104e-01  2.3339e-01  4.2824e-01
 7.3730e+00  1.4956e-02 -3.6620e-01 -1.1500e+00  4.5534e+00 -2.3112e+00
 1.2086e-01 -2.7828e+00  5.1603e+00  1.8651e+00 -5.9960e+00  1.4270e+00
-1.2016e+00  1.9237e+00  3.1299e+00  4.1164e-01 -9.8749e-01 -9.2352e-01
 3.9881e+00  1.7758e-01 -1.5232e+00 -4.8556e-01 -3.6019e+00  1.7041e+00
 1.2227e+00  3.3404e-01 -8.6912e-02  4.0270e-01  3.4648e+00  2.6652e-01
-4.3696e+00 -7.5819e-01 -9.0009e-01 -7.4193e-01  3.1780e-01 -1.6068e+00
 6.0398e-01  2.0669e+00  1.7492e-01 -1.7984e-02  6.7771e-03 -5.7923e-01
 5.6576e-01 -1.9125e-02 -1.0169e+00  3.4711e-01  4.8249e-01 -1.0198e-01
 5.0717e+00 -4.7797e+00  2.1180e+00  3.9839e-01 -2.6345e+00  2.4853e+00
-2.6006e+00  2.6319e-01  9.2093e-01  3.5440e-01 -7.9202e-01 -2.1976e+00
-4.6147e-01 -1.1240e-01  2.1185e-01  3.3967e-01 -6.7641e-02 -4.6889e-01
-1.5438e-02 -4.3545e-02 -2.3600e+00 -3.0790e-01 -4.0401e+00  5.7220e-01
 4.0342e+00 -2.4393e-02  2.2830e+00 -3.4720e-01 -1.7

1 -6.4580e-02 -2.7994e-01  1.1682e-01
 1.9693e-01 -1.9897e-01 -1.9711e-01 -9.7344e-02  1.6574e-01 -2.1547e-01
-3.8658e+00  3.2928e-01  2.2405e+00 -3.6872e-02  5.0558e-01 -3.6702e+00
 4.5865e-01  7.1968e-02 -1.4608e-01  5.6741e-02 -1.0227e-01  9.4689e-02
-4.3214e-01 -1.3818e-01 -5.8456e-01 -5.0014e-02  4.4049e-03 -4.8439e-01
-3.8468e-01 -2.3358e-01  1.5888e+00 -8.6537e-02  1.9849e-01 -2.3133e-01

Columns 31 to 36
 2.3552e-01 -1.6574e-01  8.1676e-01 -2.6972e-01  1.1256e+00 -2.0041e-01
 1.6566e+00 -1.6876e+00  2.3008e-01 -3.9939e-01 -1.9455e+00  3.5712e-01
 4.1970e-03  1.0487e-01 -6.3751e-01 -7.8106e-01  4.1384e-01 -3.8859e-04
 4.5405e-01 -1.1322e+00  9.8277e-01 -1.5992e+00  4.3767e+00 -1.1336e-01
-4.0440e+00  3.3430e-01 -2.6729e+00  4.4769e-01  3.6934e+00 -1.2583e+00
-3.7305e-02  2.1240e-02 -2.7903e-01  6.4615e-01  9.2788e-01 -1.7367e-03
 1.2405e-01  7.2195e-02  1.5661e-02 -6.3176e-02 -2.9891e-02 -1.3444e-01
-2.7005e-01  9.3573e-01  2.8339e-01  1.4859e-01 -1.0362e+00  7.0823e-02
 2.7085e

.8628e-01
-7.7697e-01 -1.7952e+00  1.3482e-03  3.5396e-01  3.6282e-01  1.1712e+00
 2.5542e+00  5.5409e+00  7.3295e-05  5.5011e-01 -1.9988e+00  8.6598e-01
-3.2071e+00 -5.9389e+00  1.5619e-01 -1.7763e-01  4.3367e-01  1.1828e+00
-7.2239e-02  3.0770e-01 -1.0359e-01 -1.4349e-01 -3.0346e-01 -4.5917e-01
 2.2404e+00  8.2638e-01  5.2116e-03  8.2808e-01  3.7751e-01 -1.2157e+00
 3.8516e+00  2.9885e+00 -1.9362e-01 -1.5087e+00 -5.4808e-01  4.5453e-01
 1.9612e+00  4.0697e-03  5.6242e-03 -1.2087e-01  6.5014e-01 -1.2530e-01
-1.3159e-01 -1.4743e-01 -1.8541e-02 -5.5873e-02  1.4543e-01  1.0323e-01
-1.8507e+00 -4.2612e-01 -3.0011e-02 -5.1358e-01  2.6536e-01  3.3413e-01
-8.8450e-02 -8.0211e-02  7.9255e-02  5.7901e-02  1.4924e-02  7.1990e-02
 4.7904e-01 -4.2809e-01 -8.0572e-02  4.9991e-02 -5.6887e-01  4.0781e-01
 1.8845e+00 -1.7800e-01  1.2770e-01 -9.1527e-01  4.3748e-01 -3.7275e-01
-5.5936e-01  4.8939e-02  3.3789e-02  3.8563e-01 -8.8682e-02  1.1046e-01
 4.1140e-01 -2.5247e-02 -1.0171e-01 -5.5698e-02  4.818

3e-03
-1.7343e-02 -4.7835e-01
-8.2888e-02 -1.7112e+00
-1.1934e-01  2.2290e+00
-3.4888e-01  4.4803e+00
-1.8560e-01 -4.6656e+00
-3.3159e-01  7.5634e-01
-2.5719e-02 -2.7233e+00
 3.1562e-02 -7.8083e-01
-2.9321e-01 -1.4487e+00
-1.4785e-01  1.2323e+00
-1.6480e-01 -1.9399e+00
-7.9581e-02  1.7855e+00
-1.0178e-02 -2.4723e-01
 2.3966e-01 -5.5779e+00
-7.5633e-02  3.5276e-01
-3.0357e-01  6.5272e-01
 7.9236e-02  3.9707e-02
-7.8949e-04  1.3647e+00
 1.2545e-02  6.7602e-02
 2.3443e-01 -1.3102e+00
 1.8142e-01  1.1711e+00
 2.2916e-02 -6.4245e-02
 7.7209e-02 -6.0592e-01
 5.6909e-03 -3.7196e-01
-5.4848e-01  6.7497e+00
-1.0527e-01 -1.5782e-01
-1.7912e-01  2.8602e-01
 3.7412e-02 -7.3083e-01
[torch.CudaTensor of size 35x50]



In [43]:
print(model)

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> (8) -> (9) -> (10) -> (11) -> (12) -> (13) -> (14) -> (15) -> (16) -> output]
  (1): nn.Sequential {
    [input -> (1) -> (2) -> output]
    (1): nn.ConcatTable {
      input
        |`-> (1): nn.SelectTable(1)
         `-> (2): nn.ConcatTable {
               input
                 |`-> (1): nn.Sequential {
                 |      [input -> (1) -> (2) -> output]
                 |      (1): nn.ConcatTable {
                 |        input
                 |          |`-> (1): nn.SelectTable(1)
                 |           `-> (2): nn.SelectTable(3)
                 |           ... -> output
                 |      }
                 |      (2): nn.Sequential {
                 |        [input -> (1) -> (2) -> (3) -> (4) -> output]
                 |        (1): nn.ParallelTable {
                 |          input
                 |            |`-> (1): nn.Linear(256 -> 50)
                 |             `-> (

tenTable
  }
  (3): nn.Sequential {
    [input -> (1) -> (2) -> output]
    (1): nn.ConcatTable {
      input
        |`-> (1): nn.SelectTable(1)
         `-> (2): nn.ConcatTable {
               input
                 |`-> (1): nn.Sequential {
                 |      [input -> (1) -> (2) -> output]
                 |      (1): nn.ConcatTable {
                 |        input
                 |          |`-> (1): nn.SelectTable(1)
                 |           `-> (2): nn.SelectTable(3)
                 |           ... -> output
                 |      }
                 |      (2): nn.Sequential {
                 |        [input -> (1) -> (2) -> (3) -> (4) -> output]
                 |        (1): nn.ParallelTable {
                 |          input
                 |            |`-> (1): nn.Linear(256 -> 50)
                 |             `-> (2): nn.Linear(35 -> 50)
                 |             ... -> output
                 |        }
                 |        (2): nn.CAddTable
 

(1)
         `-> (2): nn.ConcatTable {
               input
                 |`-> (1): nn.Sequential {
                 |      [input -> (1) -> (2) -> output]
                 |      (1): nn.ConcatTable {
                 |        input
                 |          |`-> (1): nn.SelectTable(1)
                 |           `-> (2): nn.SelectTable(3)
                 |           ... -> output
                 |      }
                 |      (2): nn.Sequential {
                 |        [input -> (1) -> (2) -> (3) -> (4) -> output]
                 |        (1): nn.ParallelTable {
                 |          input
                 |            |`-> (1): nn.Linear(256 -> 50)
                 |             `-> (2): nn.Linear(35 -> 50)
                 |             ... -> output
                 |        }
                 |        (2): nn.CAddTable
                 |        (3): nn.BatchNormalization (2D) (50)
                 |        (4): nn.ReLU
                 |      }
               

> (2) -> output]
                 |      (1): nn.ConcatTable {
                 |        input
                 |          |`-> (1): nn.SelectTable(1)
                 |           `-> (2): nn.SelectTable(3)
                 |           ... -> output
                 |      }
                 |      (2): nn.Sequential {
                 |        [input -> (1) -> (2) -> (3) -> (4) -> output]
                 |        (1): nn.ParallelTable {
                 |          input
                 |            |`-> (1): nn.Linear(256 -> 50)
                 |             `-> (2): nn.Linear(35 -> 50)
                 |             ... -> output
                 |        }
                 |        (2): nn.CAddTable
                 |        (3): nn.BatchNormalization (2D) (50)
                 |        (4): nn.ReLU
                 |      }
                 |    }
                  `-> (2): nn.Sequential {
                        [input -> (1) -> (2) -> output]
                        (1): nn.Na

) -> output]
          (1): nn.ConcatTable {
            input
              |`-> (1): nn.SelectTable(1)
               `-> (2): nn.ConcatTable {
                     input
                       |`-> (1): nn.Sequential {
                       |      [input -> (1) -> (2) -> output]
                       |      (1): nn.ConcatTable {
                       |        input
                       |          |`-> (1): nn.SelectTable(1)
                       |           `-> (2): nn.SelectTable(3)
                       |           ... -> output
                       |      }
                       |      (2): nn.Sequential {
                       |        [input -> (1) -> (2) -> (3) -> (4) -> output]
                       |        (1): nn.ParallelTable {
                       |          input
                       |            |`-> (1): nn.Linear(256 -> 50)
                       |             `-> (2): nn.Linear(35 -> 50)
                       |             ... -> output
            

...

     |`-> (1): nn.Sequential {
                             |      [input -> (1) -> (2) -> output]
                             |      (1): nn.ConcatTable {
                             |        input
                             |          |`-> (1): nn.SelectTable(1)
                             |           `-> (2): nn.SelectTable(3)
                             |           ... -> output
                             |      }
                             |      (2): nn.Sequential {
                             |        [input -> (1) -> (2) -> (3) -> (4) -> output]
                             |        (1): nn.ParallelTable {
                             |          input
                             |            |`-> (1): nn.Linear(256 -> 50)
                             |             `-> (2): nn.Linear(35 -> 50)
                             |             ... -> output
                             |        }
                             |        (2): nn.CAddTable
                       

                     |        (3): nn.BatchNormalization (2D) (50)
                       |        (4): nn.ReLU
                       |      }
                       |    }
                        `-> (2): nn.Sequential {
                              [input -> (1) -> (2) -> output]
                              (1): nn.NarrowTable
                              (2): nn.Sequential {
                                [input -> (1) -> (2) -> (3) -> (4) -> output]
                                (1): nn.ParallelTable {
                                  input
                                    |`-> (1): nn.Linear(256 -> 35)
                                     `-> (2): nn.Linear(50 -> 35)
                                     ... -> output
                                }
                                (2): nn.CAddTable
                                (3): nn.BatchNormalization (2D) (35)
                                (4): nn.ReLU
                              }
                          

e
                {
                  gradInput : table: 0x402e6180
                  input_map : table: 0x41a43140
                  _type : torch.DoubleTensor
                  output : table: 0x41a43118
                }
            }
          _type : torch.CudaTensor
          output : 
            {
              1 : CudaTensor - size: 512x256
              2 : CudaTensor - size: 512x50
              3 : CudaTensor - size: 512x35
            }
        }
      9 : 
        nn.NarrowTable
        {
          _type : torch.CudaTensor
          output : 
            {
              1 : CudaTensor - size: 512x50
              2 : CudaTensor - size: 512x35
            }
          gradInput : 
            {
              1 : CudaTensor - size: 512x256
              2 : CudaTensor - size: 512x50
              3 : CudaTensor - size: 512x35
            }
          length : 2
          offset : 2
        }
      10 : 
        nn.JoinTable
        {
          _type : torch.CudaTensor
       

          hyDesc : cdata<struct cudnnTensorStruct *[?]>: 0x402e1128
          cyDesc : cdata<struct cudnnTensorStruct *[?]>: 0x402e16b8
          _gradOutput : CudaTensor - size: 16x32x256
          seqLength : 16
          seed : 19088743
          numDirections : 1
          inputSize : 85
          inputPacked : false
          dropoutStatesSize : 1376256
          hiddenOutput : CudaTensor - size: 1x32x256
          miniBatch : 32
          mode : CUDNN_LSTM
          datatype : CUDNN_DATA_FLOAT
          weight : CudaTensor - size: 351232
          train : true
          numLayers : 1
          sync : true
          gradCellInput : CudaTensor - size: 1x32x256
          dropoutStates : CudaTensor - size: 344064
          reserve : CudaTensor - size: 655360
        }
      14 : 
        nn.Dropout(0.500000)
        {
          v2 : true
          noise : CudaTensor - size: 32x16x256
          _type : torch.CudaTensor
          p : 0.5
          gradInput : CudaTensor - size: 32x16x2

       {
          _type : torch.CudaTensor
          output : CudaTensor - size: 512x256
          gradInput : CudaTensor - size: 32x16x256
          size : LongStorage - size: 2
          numElements : 256
        }
      16 : 
        nn.Linear(256 -> 157)
        {
          gradBias : CudaTensor - size: 157
          weight : CudaTensor - size: 157x256
          _type : torch.CudaTensor
          output : CudaTensor - size: 512x157
          gradInput : CudaTensor - size: 512x256
          addBuffer : CudaTensor - size: 512
          bias : CudaTensor - size: 157
          gradWeight : CudaTensor - size: 157x256
        }
    }
  _type : torch.CudaTensor
  output : CudaTensor - size: 512x157
}


In [12]:
a = torch.zeros(2,2,2)
b = torch.ones(2,2)

In [17]:
a[2]:copy(b)

 1  1
 1  1
[torch.DoubleTensor of size 2x2]



In [18]:
a

(1,.,.) = 
  1  1
  1  1

(2,.,.) = 
  1  1
  1  1
[torch.DoubleTensor of size 2x2x2]

